# **Интерактивный тренажер по SQL. Основы реляционной модели и SQL.**  
[Курс](https://stepik.org/course/63054/syllabus) на Stepik.

Автор: **Озерова Галина Павловна**  
<img src="imgs/ozerova.jpg" title="Озерова Галина Павловна" width="200" height="200"/>

Конспектировал: **Илья Филимонов** (GiHub: [@IsFilimonov](https://github.com/IsFilimonov))  

<img src="imgs/IsFilimonov.png" title="Илья Филимонов" width="200" height="200"/>

---

# **Введение**

## От конспектирующего
(Не реклама): сам курс использует диалект MySQL, но мне комфортнее использовать PostgreSQL 12.6 ([download](https://www.postgresql.org/download/)).  
[Рекомендации](https://wiki.postgresql.org/wiki/Don%27t_Do_This) того, чего не нужно делать в Postgres.

Для работы с базой лично я использую IDE [DataGrip](https://www.jetbrains.com/ru-ru/datagrip/) (платная) или [Dbeaver](https://dbeaver.io/) (бесплатная) - под MacOS. 

Это всё сторонние решения, но можно воспользоваться более [простым способом](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259) - использовать Jupyter Notebook в роли SQL IDE. Для этого нам потребуется установить несколько библиотек. Для работы с Jupyter Notebook я использую платформу [Anaconda](https://www.anaconda.com/), поэтому все нужные библиотеки для работы я буду устанавливать именно в эту среду. Я не буду останавливаться на деталях в вопросе "как?", чтобы в деталях не закопаться. В интернете полно подсказок. Главное, должны быть установлены:
- `ipython-sql` - [пакет](https://anaconda.org/conda-forge/ipython-sql) для работы с SQL запросами;
- `psycopg2` или `psycopg2-binary` - драйвер базы данных Postgres. Там есть нюансы с установкой;
- `sqlalchemy` - ORM для работы с SQL БД.


Более подробно читайте [статью](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259), которая была выше.

Для установки соединения с БД мы единожды используем SQL Alchemy.

In [1]:
import sqlalchemy

Далее создаем соединение.

In [2]:
sqlalchemy.create_engine("postgresql://localhost:5432/postgres")

Engine(postgresql://localhost:5432/postgres)

Загружаем extension.

In [3]:
%load_ext sql

Проверяем соединение.

In [4]:
%sql postgresql://localhost:5432/postgres

'Connected: @postgres'

Чтобы узнать какая версия установлена у вас, введите в консоле запроса БД (не в командной строке, а в ячейке):

In [5]:
%%sql

SELECT version();

 * postgresql://localhost:5432/postgres
1 rows affected.


version
"PostgreSQL 12.6 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit"


Вероятнее всего у вас уже создана БД `postgres`. Будем работать с ней. В БД нам нужно создать схему `Stepik`, так сказать группировку (семантическую) для нужных нам таблиц. 

`IF NOT EXIST` означает не делать ничего (только выдать замечание), если схема с таким именем уже существует.

In [6]:
%%sql

CREATE SCHEMA IF NOT EXISTS stepik;

 * postgresql://localhost:5432/postgres
Done.


[]

## Памятка о типичных ошибках и способах их исправления

1. Приведите синтаксис запроса к общепринятому:
    - если у вас есть время, стоит изучить [руководство по стилю SQL](https://www.sqlstyle.guide/ru/);
    - можете отформатировать ваш запрос с помощью сторонних [сервисов](https://codebeautify.org/sqlformatter).
2. Проверьте, что **ключевые слова**, **названия столбцов** и **значения в ячейках**, которые необходимо найти, написаны правильно. Особенно обратите внимание, чтобы в русских названиях столбцов не было английских букв.
3. Проверьте, что:
    - количество открывающихся скобок равно количеству закрывающихся;
    - запятые разделяют перечисление столбцов, но не ключевые слова;
    - запросы разделяются точкой с запятой.
4. Последовательность команд:
```SQL
SELECT 'столбцы или * для выбора всех столбцов; обязательно'
FROM 'таблица; обязательно'
WHERE 'условие/фильтрация, например, city = 'Moscow'; необязательно'
GROUP BY 'столбец, по которому хотим сгруппировать данные; необязательно'
HAVING 'условие/фильтрация на уровне сгруппированных данных; необязательно'
ORDER BY 'столбец, по которому хотим отсортировать вывод; необязательно'
```
5. Если запрос включает подзапросы, выполните сначала подзапросы и удостоверьтесь, что получаете ожидаемый результат.
6. **Важно!** На платформе Stepik используется MySQL, версия 8.0.21.

# **Основные понятия реляционных баз данных**

Реляционная модель была разработана в конце 1960-х годов Е.Ф.Коддом. Она определяет способ представления данных (структуру данных), методы защиты данных (целостность данных), и операции, которые можно выполнять с данными (манипулирование данными). Эта модель лежит в основе всех реляционных баз данных до настоящего времени.

**Основные принципы реляционных баз данных:**

- Все данные на концептуальном уровне представляются в виде объектов, заданных в виде строк и столбцов, называемых отношением, более распространенное название – таблица;
- В пересечение строки и столбца таблицы можно занести только одно значение;
- Все операции выполняются над целыми отношениями и результатом этих операций является отношение.

Пример отношения: 
<center><img src="imgs/s1.jpg" title="slide 1" width="75%" height="75%"/></center>

На примере таблицы **Сотрудник** рассмотрим терминологию реляционных баз данных:

- **Отношение** (*relation*) – это структура данных целиком, набор записей (в обычном понимании – **таблица**) , в  примере – это `Сотрудник`;
- **Кортеж** (*tuple*) – это каждая строка, содержащая данные (более распространенный термин – **запись**), например, `<001, Борин С.А, 234-01-23, программист>`, все кортежи в отношении должны быть различны;
- **Мощность** – число кортежей в таблице (проще говоря, **число записей**), в данном случае 3, мощность отношения может быть любой (от 0 до бесконечности), порядок следования кортежей - неважен;
- **Атрибут** (*attribute*) – это столбец в таблице (более распространенный термин – **поле**), в примере – `Табельный номер`, `Фамилия И.О.`, `Телефон`, `Должность`); 
- **Размерность** – это число атрибутов в таблице, в данном случае – 4; размерность отношения должна быть больше 0, порядок следования атрибутов существенен;
- **Домен атрибута** (*domain*) – это допустимые значения (неповторяющиеся), которые можно занести в поле , например для атрибута `Должность` домен – `{инженер, программист}`.

## Отношение, реляционная модель

**Рассмотрим пример**: некоторый склад, на котором хранятся книги. Известно название книги, ее автор, количество экземпляров на складе и ее цена.

Всю эту информацию можно представить в виде таблицы, состоящей из 4 столбцов (приведено только 4 записи, на самом деле их значительно больше):

| Название           | Автор            | Цена, руб | Количество |
|--------------------|------------------|-----------|------------|
| Мастер и Маргарита | Булгаков М.А.    | 670.99    | 3          |
| Белая гвардия      | Булгаков М.А.    | 540.50    | 5          |
| Идиот              | Достоевский Ф.М. | 460       | 10         |
| Братья Карамазовы  | Достоевский Ф.М. | 799.01    | 2          |

Перед созданием таблицы в базе данных необходимо описать ее структуру. Для этого выполняется следующая последовательность шагов:

1. Дать таблице имя, пусть она будет называться `book`, вот некоторые **правила для выбора имен таблиц**:

- может включать английские буквы, цифры и знак подчеркивания, должно начинаться с буквы;
- имя должно быть уникальным в пределах базы данных.

Также **рекомендуется**:

- чтобы имя было существительным в единственном числе;
- имя должно быть понятным и соответствовать тому объекту, который оно описывает;
- имя должно быть как можно короче, максимум до 10 символов.

**Важно**. Имена таблиц являются регистрозависимыми из-за операционной системы на которой работает stepik, то есть имя `book` и `Book` – разные имена. Рекомендуется для записи имен таблиц использовать только строчные (маленькие) буквы.

2. Определить структуру таблицы, из каких атрибутов(столбцов, полей) она будет состоять,  в нашем случае это:

- `title` – поле для хранения названия книги;
- `author` – поле с фамилией автора книги ;
- `priсe` – цена книги;
- `amount` – количество книг.

**Правила по выбору имени поля информационного объекта**:

- может включать английские буквы, цифры и знак подчеркивания, должно начинаться с буквы;
- имя поля должно быть уникальным в пределах таблицы.

**Рекомендации по выбору имени поля информационного объекта**:

- имя должно быть понятным и соответствовать тем данным, которые хранятся в поле;
- имя может состоять из нескольких слов, тогда слова разделяются подчеркиванием, после подчеркивания слово пишется с маленькой буквы.

3. Включить ключевое поле `book_id`, которое является ОБЯЗАТЕЛЬНЫМ ЭЛЕМЕНТОМ каждой реляционной таблицы. Ключевое поле является уникальным для каждой записи, однозначно определяет запись и в дальнейшем будет использоваться для связей с другими таблицами.

**Рекомендации по именованию  ключевых полей**:

- имя должно состоять  из двух частей: начинаться с названия таблицы, которой поле принадлежит,   затем через подчеркивание необходимо указать `id`.

Таким образом, наша таблицаbookбудет выглядеть следующим образом:

| book_id | title              | author           | price  | amount |
|---------|--------------------|------------------|--------|--------|
| 1       | Мастер и Маргарита | Булгаков М.А.    | 670.99 | 3      |
| 2       | Белая гвардия      | Булгаков М.А.    | 540.50 | 5      |
| 3       | Идиот              | Достоевский Ф.М. | 460    | 10     |
| 4       | Братья Карамазовы  | Достоевский Ф.М. | 799.01 | 2      |

## Выбор типов данных для полей 

- `INT`, `INTEGER` - Целое число. Значения от -2 147 483 648$ до $2 147 483 647.
- `DECIMAL`, `NUMERIC` - Вещественное число. Диапазон значений $(-10^{38}+1)$ до $(10^{38}+1)$.
- `DATE` - Дата. Дата в формате ГГГГ-ММ-ДД (2020-07-26).
- `VARCHAR` - Строка длиной 255 символов.

**Рекомендации по выбору типов данных для полей таблицы**:

- Выбирайте минимальный тип данных исходя из максимального значения поля. Например, если максимальный текст, который может быть записан в поле, имеет длину 25 символов, значит нужно использовать тип `VARCHAR(25)`.
- Для описания ключевого поля используйте описание `INT PRIMARY KEY AUTO_INCREMENT`. Это значит, что в поле будут заноситься различные целые числа, при этом они будут автоматически генерироваться (каждая следующая строка будет иметь значение ключа на 1 больше предыдущего).

Определим тип данных для каждого поля таблицы **book**:
- `book_id` - ключевой столбец, целое число, которое должно генерироваться автоматически  - `INT PRIMARY KEY AUTO_INCREMENT`;
- `title` - строка текста, ее длина выбирается в зависимости от данных, которые предполагается хранить в поле, предположим, что название книги не превышает 50 символов - `VARCHAR(50)`;
- `author` - строка текста - `VARCHAR(30)`;
- `priсe` - для описание денежного значения используется числовой тип данных с двумя знаками после запятой - `DECIMAL(8,2)`;
- `amount` - целое число - `INT`.



## Создание таблицы

**Внимание**! Все блоки кода ниже будет представлены на языке курса MySQL, но так же продублириованы на PostgreSQL.

Пример создания таблицы:

```MYSQL
/*MySQL*/
CREATE TABLE genre(
    genre_id INT PRIMARY KEY AUTO_INCREMENT, 
    name_genre VARCHAR(30)
);
```

**Рекомендации по записи SQL запроса**:

- **Ключевые слова**: SQL не является регистрозависимым языком (CREATE и create - одно и тоже ключевое слово). 
- **Ключевые слова SQL** и **типы данных** рекомендуется  записывать прописными (большими) буквами.
- **Имена таблиц** и **полей** - строчными (маленькими) буквами.
- SQL-запрос можно писать на нескольких строках.
- В конце SQL-запроса ставится точка с запятой (хотя если Вы пишете один запрос, это необязательно).

In [7]:
%%sql
/*pgSQL*/
CREATE TABLE IF NOT EXISTS stepik.book (
  book_id int primary key generated always as identity, 
  title varchar(50), 
  author varchar(30), 
  price decimal(8, 2), 
  amount int
);

 * postgresql://localhost:5432/postgres
Done.


[]

## Вставка записи в таблицу

Пример добавления записей в таблицу:
```MySQL
INSERT INTO таблица(поле1, поле2) 
VALUES (значение1, значение2);
```

В результате выполнения запроса новая запись заносится в конец обновляемой таблицы.

**Необходимо учитывать**:
- количество полей и количество значений в списках должны совпадать;
- должно существовать прямое соответствие между позицией одного и того же элемента в обоих списках, поэтому первый элемент списка значений должен относиться к первому столбцу в списке столбцов, второй – ко второму столбцу и т.д.;
- типы данных элементов в списке значений должны быть совместимы с типами данных соответствующих столбцов таблицы ( целое число можно занести в поле типа `DECIMAL`, обратная операция - недопустима);
- новые значения нельзя добавлять в поля, описанные как `PRIMARY KEY AUTO_INCREMENT`;
- рекомендуется заполнять все поля записи, если же поле пропущено, значение этого поля зависит от установленных по умолчанию значений, если значения не установлены - на данной платформе вставляется пустое значение (`NULL`).

### Задача 1
```MySQL
INSERT INTO book(title, author, price, amount) 
VALUES 
  (
    "Мастер и Маргарита", 
    "Булгаков М.А.", 
    670.99, 
    3
  );
```

В Postgres для обозначения строки используются одинарные кавычки (`''`), а двойные для идентификаторов, а так же, чтобы снять вопросы чувствительности регистра. Почитайте к примеру [тут](https://www.prisma.io/dataguide/postgresql/short-guides/quoting-rules) или в [документации](https://www.postgresql.org/docs/12/sql-syntax-lexical.html). 

In [8]:
%%sql
INSERT INTO stepik.book(title, author, price, amount)
VALUES ('Мастер и Маргарита', 'Булгаков М.А.', 670.99, 3);

 * postgresql://localhost:5432/postgres
1 rows affected.


[]

### Задача 2
```MySQL
INSERT INTO book(title, author, price, amount) 
VALUES 
  ("Белая гвардия", "Булгаков М.А.", 540.50, 5),
  ("Идиот", "Достоевский Ф.М.", 460.00, 10),
  ("Братья Карамазовы", "Достоевский Ф.М.", 799.01, 2);
```

In [9]:
%%sql
INSERT INTO stepik.book(title, author, price, amount)
VALUES 
('Белая гвардия', 'Булгаков М.А.', 540.50, 5),
('Идиот', 'Достоевский Ф.М.', 460.00, 10),
('Братья Карамазовы', 'Достоевский Ф.М.', 799.01, 2);

 * postgresql://localhost:5432/postgres
3 rows affected.


[]

### Дополнение

Странно, но отсутствовало задание на добавление записи Есенина в таблицу, хотя далее эта запись присутствует. Добавим самостоятельно.

In [10]:
%%sql
INSERT INTO stepik.book(title, author, price, amount)
VALUES ('Стихотворения и поэмы', 'Есенин С.А.', 650.00, 15);

 * postgresql://localhost:5432/postgres
1 rows affected.


[]

## Выборка данных

### Выборка всех данных из таблицы

Пример выборки всех записей из таблицы:

```MySQL
SELECT * FROM book;
```

In [11]:
%%sql
SELECT * FROM stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


book_id,title,author,price,amount
1,Мастер и Маргарита,Булгаков М.А.,670.99,3
2,Белая гвардия,Булгаков М.А.,540.50,5
3,Идиот,Достоевский Ф.М.,460.00,10
4,Братья Карамазовы,Достоевский Ф.М.,799.01,2
5,Стихотворения и поэмы,Есенин С.А.,650.00,15


### Выборка отдельных столбцов

Пример выборки записей по нескольким столбцам из таблицы:

```MySQL
SELECT author, title, price
FROM book;
```

In [12]:
%%sql
SELECT author, title, price
FROM stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


author,title,price
Булгаков М.А.,Мастер и Маргарита,670.99
Булгаков М.А.,Белая гвардия,540.50
Достоевский Ф.М.,Идиот,460.00
Достоевский Ф.М.,Братья Карамазовы,799.01
Есенин С.А.,Стихотворения и поэмы,650.00


### Выборка отдельных столбцов и присвоение им имен

Пример выборки записей по нескольким столбцам из таблицы с присвоением имен столбцам:

```MySQL
SELECT 
    title AS "Название",
    author AS "Автор"
FROM
    book;
```

In [13]:
%%sql
SELECT 
    title AS "Название",
    author AS "Автор"
FROM 
    stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


Название,Автор
Мастер и Маргарита,Булгаков М.А.
Белая гвардия,Булгаков М.А.
Идиот,Достоевский Ф.М.
Братья Карамазовы,Достоевский Ф.М.
Стихотворения и поэмы,Есенин С.А.


### Выборка данных с созданием вычисляемого столбца

Для упаковки каждой книги требуется 1 лист бумаги, цена которого 1 рубль 65 копеек. Посчитать стоимость упаковки для каждой книги (сколько денег потребуется, чтобы упаковать все экземпляры книги). В запросе вывести название книги, ее количество и стоимость упаковки, последний столбец назвать `pack`.

```MySQL
SELECT
    title,
    amount,
    amount * 1.65 AS "pack"
FROM
    book;
```

In [14]:
%%sql
SELECT 
    title,
    amount,
    amount * 1.65 AS "pack"
FROM 
    stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


title,amount,pack
Мастер и Маргарита,3,4.95
Белая гвардия,5,8.25
Идиот,10,16.50
Братья Карамазовы,2,3.30
Стихотворения и поэмы,15,24.75


### Выборка данных, вычисляемые столбцы, математические функции

- `CEILING(x)` - возвращает наименьшее целое число, большее или равное x (округляет до целого числа в большую сторону).
```MySQL
CEILING(4.2)=5
CEILING(-5.8)=-5
```

- `ROUND(x, k)` - округляет значение x до k знаков после запятой, если k не указано – x округляется до целого.
```MySQL
ROUND(4.361)=4
ROUND(5.86592,1)=5.9
```

- `FLOOR(x)` - возвращает наибольшее целое число, меньшее или равное x (округляет до  целого числа в меньшую сторону.
```MySQL
FLOOR(4.2)=4
FLOOR(-5.8)=-6
```

- `POWER(x, y)` - возведение x в степень y.
```MySQL
POWER(3,4)=81.0
```

- `SQRT(x)` - квадратный корень из x.
```MySQL
SQRT(4)=2.0
SQRT(2)=1.41...
```

- `DEGREES(x)` - конвертирует значение x из радиан в градусы.
```MySQL
DEGREES(3) = 171.8...
```

- `RADIANS(x)` - конвертирует значение x из градусов в радианы.
```MySQL
RADIANS(180)=3.14...
```

- `ABS(x)` - модуль числа x.
```MySQL
ABS(-1) = 1
ABS(1) = 1
```

- `PI()` - число $\pi$.
```MySQL
pi = 3.1415926...	
```

**Пояснение**. Существуют разные способы округления чисел. В SQL реализовано математическое округление. Для округления вещественного числа нужно в записи числа выбрать разряд в дробной части, до которого производится округление. Цифра, записанная в выбранном разряде: не меняется, если следующая за ней справа цифра - 0, 1, 2, 3 или 4; увеличивается на единицу, если следующая за ней справа цифра - 5,6,7,8 или 9.

#### Задача

В конце года цену всех книг на складе пересчитывают – снижают ее на 30%. Написать SQL запрос, который из таблицы **book** выбирает `названия`, `авторов`, `количества` и вычисляет новые цены книг. Столбец с новой ценой назвать `new_price`, цену округлить до 2-х знаков после запятой.

```MySQL
SELECT
    title,
    author,
    amount,
    ROUND(price * 0.7, 2) AS "new_price"
FROM
    book;
```

In [15]:
%%sql
SELECT
    title,
    author,
    amount,
    ROUND(price * 0.7, 2) AS "new_price"
FROM
    stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


title,author,amount,new_price
Мастер и Маргарита,Булгаков М.А.,3,469.69
Белая гвардия,Булгаков М.А.,5,378.35
Идиот,Достоевский Ф.М.,10,322.00
Братья Карамазовы,Достоевский Ф.М.,2,559.31
Стихотворения и поэмы,Есенин С.А.,15,455.00


### Выборка данных, вычисляемые столбцы, логические функции

```MySQL
IF(логическое_выражение, истина, ложь)
```

Усложним вычисление скидки в зависимости от количества книг. Если количество книг меньше 4 – то скидка 50%, меньше 11 – 30%, в остальных случаях – 10%. И еще укажем какая именно скидка на каждую книгу.

```MySQL
SELECT title, amount, price,
    ROUND(IF(amount < 4, price * 0.5, IF(amount < 11, price * 0.7, price * 0.9)), 2) AS sale,
    IF(amount < 4, 'скидка 50%', IF(amount < 11, 'скидка 30%', 'скидка 10%')) AS Ваша_скидка
FROM book;
```

#### Задача

При анализе продаж книг выяснилось, что наибольшей популярностью пользуются книги Михаила Булгакова, на втором месте книги Сергея Есенина. Исходя из этого решили поднять цену книг Булгакова на 10%, а цену книг Есенина - на 5%. Написать запрос, куда включить `автора`, `название книги` и `новую цену`, последний столбец назвать `new_price`. Значение округлить до двух знаков после запятой.

Пояснение:

- фамилию автора задавать с инициалами (как занесено в таблице), заключая в одинарные или двойные кавычки;
- для сравнения на равенство использовать знак `=`, например `author="Булгаков М.А."`.

```MySQL
SELECT
    author,
    title,
    ROUND(
        IF (author = "Булгаков М.А.", price * 1.1, (
            IF (author="Есенин С.А.", price * 1.05, price))), 2) AS "new_price"
FROM
    book;
```

**Стоит пояснить:**  
в Postgres не используется `IF`, вместо этого [используют](https://www.postgresql.org/docs/12/functions-conditional.html) `CASE ... WHEN ... THEN ... ELSE ... END`.

In [16]:
%%sql
SELECT 
  author, 
  title, 
  ROUND(
    CASE 
        WHEN author = 'Булгаков М.А.' THEN price * 1.1 
        WHEN author = 'Есенин С.А.' THEN price * 1.05 
        ELSE price END, 
    2
  ) AS new_price 
FROM 
  stepik.book;

 * postgresql://localhost:5432/postgres
5 rows affected.


author,title,new_price
Булгаков М.А.,Мастер и Маргарита,738.09
Булгаков М.А.,Белая гвардия,594.55
Достоевский Ф.М.,Идиот,460.00
Достоевский Ф.М.,Братья Карамазовы,799.01
Есенин С.А.,Стихотворения и поэмы,682.50
